In [ ]:
pip install loguru

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.0 MB/s eta 0:00:00


In [ ]:
pip install pendulum

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 kB 7.8 MB/s eta 0:00:00


In [ ]:
pip install cufflinks

  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install lightgbm

# Feature Engineering

In [ ]:
import os
import shutil
import pprint
from datetime import datetime
import pandas as pd
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import SGDClassifier
import plotly.offline as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from sklearn.model_selection import train_test_split
init_notebook_mode(connected=True)
import cufflinks as cf
cf.go_offline()
import pickle
import gc
import lightgbm as lgb
warnings.filterwarnings('ignore')
%matplotlib inline
import plotly.io as pio
pio.renderers.default = "colab"
from loguru import logger
import pendulum
import sys

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning:


Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.




In [ ]:
# Set logging format
log_fmt = "<green>{time:YYYY-MM-DD HH:mm:ss.SSS!UTC}</green> | <level>{level: <8}</level> | <cyan>{name}</cyan>:<cyan>{function}</cyan>:<cyan>{line}</cyan> - {message}"
logger.configure(handlers=[{"sink": sys.stderr, "format": log_fmt}])

# current date
CURRENT_DATE = pendulum.now(tz="UTC")

In [ ]:
DIR = "/content/drive/MyDrive/home-credit-default-risk/"

In [ ]:
CLEANED_DATA_DIR = "/content/drive/MyDrive/cleaned_data/"

In [ ]:
import numpy as np
import pandas as pd
from loguru import logger

def reduce_memory_usage(df: pd.DataFrame) -> pd.DataFrame:
    """
    Optimize the memory usage of a DataFrame by downcasting numerical columns to more efficient types.

    Args:
        df (pd.DataFrame): The DataFrame for which memory usage should be optimized.

    Returns:
        pd.DataFrame: The DataFrame with optimized memory usage.

    Notes:
        - This function will downcast integer columns to the smallest possible integer type (int8, int16, int32, or int64)
          based on their minimum and maximum values.
        - It will also downcast floating-point columns to the smallest possible floating-point type (float16, float32, or float64).
        - Columns of type object (e.g., strings) are not modified.
        - The function prints the memory usage before and after optimization and the percentage decrease in memory usage.
    """
    # Calculate and print initial memory usage
    start_mem = df.memory_usage().sum() / 1024**2
    logger.info('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    # Iterate through each column to optimize its data type
    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()

            if str(col_type)[:3] == 'int':
                # Downcast integer columns
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                # Downcast float columns
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    # Calculate and print memory usage after optimization
    end_mem = df.memory_usage().sum() / 1024**2
    logger.info('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    logger.info('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

# Preprocessing of each dataset

## Bureau and Bureau_balance

In [ ]:
import pickle
import numpy as np
import pandas as pd
from loguru import logger
import pendulum
from typing import Optional, Union

class preprocess_bureau_balance_and_bureau:
    '''
    Preprocess the tables bureau_balance and bureau.
    Contains 4 member functions:
        1. init method
        2. preprocess_bureau_balance method
        3. preprocess_bureau method
        4. main method
    '''

    def __init__(self, file_directory: str = '', verbose: bool = True, dump_to_pickle: bool = False):
        '''
        This function is used to initialize the class members

        Inputs:
            self
            file_directory: Path, str, default = ''
                The path where the file exists. Include a '/' at the end of the path in input
            verbose: bool, default = True
                Whether to enable verbosity or not
            dump_to_pickle: bool, default = False
                Whether to pickle the final preprocessed table or not

        Returns:
            None
        '''

        self.file_directory = file_directory
        self.verbose = verbose
        self.dump_to_pickle = dump_to_pickle
        self.start = datetime.now()
        logger.info('Preprocessing class initialized.')

    def preprocess_bureau_balance(self):
        '''
        Function to preprocess bureau_balance table.
        This function first loads the table into memory, does some feature engineering, and finally
        aggregates the data over SK_ID_BUREAU

        Inputs:
            self

        Returns:
            preprocessed and aggregated bureau_balance table.
        '''

        if self.verbose:
            logger.info('#######################################################')
            logger.info('#          Pre-processing bureau_balance.csv          #')
            logger.info('#######################################################')
            logger.info("\nLoading the DataFrame, bureau_balance.csv, into memory...")

        bureau_balance = reduce_memory_usage(pd.read_csv(self.file_directory + 'bureau_balance.csv'))

        if self.verbose:
            logger.info("Loaded bureau_balance.csv")
            logger.info(f"Time Taken to load = {datetime.now() - self.start}")
            logger.info("\nStarting Data Cleaning and Feature Engineering...")

        # # Encode STATUS labels
        # dict_for_status = {'C': 0, '0': 1, '1': 2, '2': 3, 'X': 4, '3': 5, '4': 6, '5': 7}
        # bureau_balance['STATUS'] = bureau_balance['STATUS'].map(dict_for_status)

        # if self.verbose:
        #     logger.info("Halfway through. A little bit more patience...")
        #     logger.info(f"Total Time Elapsed = {datetime.now() - self.start}")

        # # Aggregating over whole dataset
        # aggregated_bureau_balance = bureau_balance.groupby(['SK_ID_BUREAU']).mean().reset_index()

        if self.verbose:
            logger.info('Done preprocessing bureau_balance.')
            logger.info(f"\nInitial Size of bureau_balance: {bureau_balance.shape}")
            logger.info(f'Size of bureau_balance after Pre-Processing, Feature Engineering and Aggregation: {bureau_balance.shape}')
            logger.info(f'\nTotal Time Taken = {datetime.now() - self.start}')

        if self.dump_to_pickle:
            if self.verbose:
                logger.info('\nPickling pre-processed bureau_balance to bureau_balance_preprocessed.pkl')
            with open(self.file_directory + 'bureau_balance_preprocessed.pkl', 'wb') as f:
                pickle.dump(aggregated_bureau_balance, f)
            if self.verbose:
                logger.info('Done.')

        return bureau_balance

    def preprocess_bureau(self, aggregated_bureau_balance):
        '''
        Function to preprocess the bureau table and merge it with the aggregated bureau_balance table.

        Inputs:
            self
            aggregated_bureau_balance: DataFrame of aggregated bureau_balance table

        Returns:
            Final preprocessed, merged and aggregated bureau table
        '''

        if self.verbose:
            start2 = datetime.now()
            logger.info('\n##############################################')
            logger.info('#          Pre-processing bureau.csv         #')
            logger.info('##############################################')
            if self.verbose:
                start2 = datetime.now()
                logger.info('Starting preprocessing of bureau.csv')
            logger.info("\nLoading the DataFrame, bureau.csv, into memory...")

        bureau = reduce_memory_usage(pd.read_csv(self.file_directory + 'bureau.csv'))

        if self.verbose:
            logger.info("Loaded bureau.csv")
            logger.info(f"Time Taken to load = {datetime.now() - start2}")
            logger.info("\nStarting Data Cleaning and Feature Engineering...")
        # Merge with aggregated_bureau_balance
        bureau_merged = bureau.merge(aggregated_bureau_balance, on=['SK_ID_BUREAU'], how='right').drop('SK_ID_BUREAU', axis=1)
        # Combine numerical features
        bureau_numerical_aggregated = bureau_merged.select_dtypes(include=[np.number]).groupby(['SK_ID_CURR']).mean().reset_index()
        bureau_numerical_aggregated.columns = ['BUREAU_' + column if column != 'SK_ID_CURR' else column for column in bureau_numerical_aggregated.columns]

        # Combine categorical features
        bureau_categorical = pd.get_dummies(bureau_merged.select_dtypes('object'))
        bureau_categorical['SK_ID_CURR'] = bureau['SK_ID_CURR']
        bureau_categorical_aggregated = bureau_categorical.groupby(['SK_ID_CURR']).mean().reset_index()


        # Merge numerical and categorical features
        bureau_merged_aggregated = bureau_numerical_aggregated.merge(bureau_categorical_aggregated, on='SK_ID_CURR')
        bureau_merged_aggregated.update(bureau_merged_aggregated.fillna(0))
        bureau_merged_aggregated.columns = ['BUREAU_' + column if column != 'SK_ID_CURR' else column for column in bureau_merged_aggregated.columns]

        bureau_merged_aggregated.fillna(0, inplace=True)

        if self.verbose:
            logger.info('Preprocessing of bureau completed.')
            logger.info('Initial Size of bureau: {}', bureau.shape)
            logger.info('Size after merging, preprocessing, and aggregation: {}', bureau_merged_aggregated.shape)
            logger.info('Total Time Taken: {}', datetime.now() - self.start)

        if self.dump_to_pickle:
            if self.verbose:
                logger.info('Pickling pre-processed bureau and bureau_balance to bureau_merged_preprocessed.pkl')
            with open(self.file_directory + 'bureau_merged_preprocessed.pkl', 'wb') as f:
                pickle.dump(bureau_merged_aggregated, f)
            if self.verbose:
                logger.info('Pickling completed.')
        if self.verbose:
            logger.info('-' * 100)

        return bureau_merged_aggregated

    def main(self) -> pd.DataFrame:
        '''
        Function to be called for complete preprocessing and aggregation of the bureau and bureau_balance tables.

        Inputs:
            self

        Returns:
            pd.DataFrame: The final preprocessed and merged `bureau` and `bureau_balance` tables.
        '''

        # Preprocess the bureau_balance first
        aggregated_bureau_balance = self.preprocess_bureau_balance()

        # Preprocess the bureau table next, by combining it with the aggregated bureau_balance
        bureau_merged_aggregated = self.preprocess_bureau(aggregated_bureau_balance)


        return bureau_merged_aggregated


In [ ]:
bureau_aggregated = preprocess_bureau_balance_and_bureau(file_directory=DIR).main()

2024-08-06 22:33:27.246 | INFO     | __main__:__init__:39 - Preprocessing class initialized.
2024-08-06 22:33:27.260 | INFO     | __main__:preprocess_bureau_balance:55 - #######################################################
2024-08-06 22:33:27.262 | INFO     | __main__:preprocess_bureau_balance:56 - #          Pre-processing bureau_balance.csv          #
2024-08-06 22:33:27.264 | INFO     | __main__:preprocess_bureau_balance:57 - #######################################################
2024-08-06 22:33:27.268 | INFO     | __main__:preprocess_bureau_balance:58 - 
Loading the DataFrame, bureau_balance.csv, into memory...
2024-08-06 22:33:47.543 | INFO     | __main__:reduce_memory_usage:24 - Memory usage of dataframe is 624.85 MB
2024-08-06 22:33:47.762 | INFO     | __main__:reduce_memory_usage:55 - Memory usage after optimization is: 338.46 MB
2024-08-06 22:33:47.763 | INFO     | __main__:reduce_memory_usage:56 - Decreased by 45.8%
2024-08-06 22:33:47.766 | INFO     | __main__:preproces

## Previous Application

In [ ]:
class preprocess_previous_application:
    '''
    Preprocess the previous_application table.

    This class contains methods to load, clean, preprocess, and aggregate the `previous_application` table.

    Attributes:
        file_directory (str): Path to the directory containing the data files.
        verbose (bool): Whether to enable verbose logging.
        dump_to_pickle (bool): Whether to pickle the final preprocessed table.
        nrows (Optional[int]): Number of rows to read from the CSV file.
    '''

    def __init__(self, file_directory: str = '', verbose: bool = True, dump_to_pickle: bool = False, nrows: Optional[int] = None):
        '''
        Initializes the preprocess_previous_application class.

        Args:
            file_directory (str): Path to the directory where the files are located.
            verbose (bool): Whether to enable verbose logging.
            dump_to_pickle (bool): Whether to pickle the final preprocessed table.
            nrows (Optional[int]): Number of rows to read from the CSV file.
        '''
        self.file_directory = file_directory
        self.verbose = verbose
        self.dump_to_pickle = dump_to_pickle
        self.nrows = nrows
        self.start = datetime.now()
        logger.info('Preprocessing class initialized.')

    def load_dataframe(self):
        '''
        Loads the `previous_application.csv` DataFrame into memory.

        Returns:
            None
        '''
        if self.verbose:
            logger.info('########################################################')
            logger.info('#        Pre-processing previous_application.csv        #')
            logger.info('########################################################')
            logger.info("Loading the DataFrame, previous_application.csv, into memory...")

        # Loading the DataFrame into memory
        self.previous_application = reduce_memory_usage(pd.read_csv(self.file_directory + 'previous_application.csv', nrows=self.nrows))
        self.initial_shape = self.previous_application.shape

        if self.verbose:
            logger.info("Loaded previous_application.csv")
            logger.info('Time Taken to load: {}', datetime.now() - self.start)

    def data_cleaning(self):
        '''
        Cleans the data by removing erroneous points and filling categorical NaNs with 'XNA'.

        Returns:
            None
        '''
        if self.verbose:
            start = datetime.now()
            logger.info('Starting Data Cleaning...')

        # Example of data cleaning (customize as needed)
        # self.previous_application['column_name'].fillna('XNA', inplace=True)

        if self.verbose:
            logger.info("Data Cleaning Done.")
            logger.info('Time taken: {}', datetime.now() - start)

    def preprocessing_feature_engineering(self):
        '''
        Performs preprocessing such as categorical encoding and feature engineering.

        Returns:
            None
        '''
        if self.verbose:
            start = datetime.now()
            logger.info("Performing Preprocessing and Feature Engineering...")

        # Example of feature engineering (customize as needed)
        # self.previous_application['new_feature'] = self.previous_application['feature_1'] / self.previous_application['feature_2']

        if self.verbose:
            logger.info("Preprocessing and Feature Engineering Done.")
            logger.info('Time taken: {}', datetime.now() - start)

    def aggregations(self) -> pd.DataFrame:
        '''
        Aggregates the previous applications over `SK_ID_CURR` and merges with application_bureau.

        Returns:
            pd.DataFrame: Final DataFrame after merging and aggregations.
        '''
        if self.verbose:
            logger.info("Aggregating previous applications over SK_ID_CURR...")

        # Number of previous applications per customer
        grp = self.previous_application[['SK_ID_CURR', 'SK_ID_PREV']].groupby(by=['SK_ID_CURR'])['SK_ID_PREV'].count().reset_index().rename(columns={'SK_ID_PREV': 'PREV_APP_COUNT'})
        self.previous_application = self.previous_application.merge(grp, on=['SK_ID_CURR'], how='right')

        # Combining numerical features
        previous_numerical_aggregated = self.previous_application.select_dtypes(include=[np.number]).drop('SK_ID_PREV', axis=1).groupby(by=['SK_ID_CURR']).mean().reset_index()

        # Combining categorical features
        previous_categorical = pd.get_dummies(self.previous_application.select_dtypes('object'))
        previous_categorical['SK_ID_CURR'] = self.previous_application['SK_ID_CURR']
        previous_categorical_aggregated = previous_categorical.groupby('SK_ID_CURR').mean().reset_index()

        # Merge numerical and categorical features
        previous_aggregated = previous_numerical_aggregated.merge(previous_categorical_aggregated, on='SK_ID_CURR')
        previous_aggregated.columns = ['PREV_' + column if column != 'SK_ID_CURR' else column for column in previous_aggregated.columns]
        previous_aggregated.fillna(0, inplace=True)

        if self.verbose:
            logger.info('Aggregations Done.')
            logger.info('Size after merging, preprocessing, and aggregation: {}', previous_aggregated.shape)

        return previous_aggregated

    def main(self) -> pd.DataFrame:
        '''
        Performs complete preprocessing and aggregation of the `previous_application` table.

        Returns:
            pd.DataFrame: Final preprocessed and aggregated `previous_application` table.
        '''
        # Loading the DataFrame
        self.load_dataframe()

        # Cleaning the data
        self.data_cleaning()

        # Preprocessing the categorical features and creating new features
        self.preprocessing_feature_engineering()

        # Aggregating data over SK_ID_CURR and merging with application_bureau
        previous_aggregated = self.aggregations()

        if self.verbose:
            logger.info('Done aggregations.')
            logger.info('Initial Size of previous_application: {}', self.initial_shape)
            logger.info('Size of previous_application after Pre-Processing, Feature Engineering and Aggregation: {}', previous_aggregated.shape)
            logger.info('Total Time Taken: {}', datetime.now() - self.start)

        if self.dump_to_pickle:
            if self.verbose:
                logger.info('Pickling pre-processed previous_application to previous_application_preprocessed.pkl')
            with open(self.file_directory + 'previous_application_preprocessed.pkl', 'wb') as f:
                pickle.dump(previous_aggregated, f)
            if self.verbose:
                logger.info('Pickling completed.')

        return previous_aggregated

In [ ]:
previous_aggregated = preprocess_previous_application(file_directory=CLEANED_DATA_DIR).main()

2024-08-06 22:34:45.165 | INFO     | __main__:__init__:29 - Preprocessing class initialized.
2024-08-06 22:34:45.167 | INFO     | __main__:load_dataframe:39 - ########################################################
2024-08-06 22:34:45.170 | INFO     | __main__:load_dataframe:40 - #        Pre-processing previous_application.csv        #
2024-08-06 22:34:45.172 | INFO     | __main__:load_dataframe:41 - ########################################################
2024-08-06 22:34:45.174 | INFO     | __main__:load_dataframe:42 - Loading the DataFrame, previous_application.csv, into memory...
2024-08-06 22:34:57.433 | INFO     | __main__:reduce_memory_usage:24 - Memory usage of dataframe is 471.48 MB
2024-08-06 22:34:58.127 | INFO     | __main__:reduce_memory_usage:55 - Memory usage after optimization is: 293.08 MB
2024-08-06 22:34:58.128 | INFO     | __main__:reduce_memory_usage:56 - Decreased by 37.8%
2024-08-06 22:34:58.133 | INFO     | __main__:load_dataframe:49 - Loaded previous_applicat

## installments_payments

In [ ]:
class preprocess_installments_payments:
    '''
    Preprocess the installments_payments table.

    This class contains methods to load, preprocess, and aggregate the `installments_payments` table.

    Attributes:
        file_directory (str): Path to the directory containing the data files.
        verbose (bool): Whether to enable verbose logging.
        dump_to_pickle (bool): Whether to pickle the final preprocessed table.
        nrows (Optional[int]): Number of rows to read from the CSV file.
    '''

    def __init__(self, file_directory: str = '', verbose: bool = True, dump_to_pickle: bool = False, nrows: Optional[int] = None):
        '''
        Initializes the preprocess_installments_payments class.

        Args:
            file_directory (str): Path to the directory where the files are located.
            verbose (bool): Whether to enable verbose logging.
            dump_to_pickle (bool): Whether to pickle the final preprocessed table.
            nrows (Optional[int]): Number of rows to read from the CSV file.
        '''
        self.file_directory = file_directory
        self.verbose = verbose
        self.dump_to_pickle = dump_to_pickle
        self.nrows = nrows
        self.start = datetime.now()
        logger.info('Preprocessing class initialized.')

    def load_dataframe(self):
        '''
        Loads the `installments_payments.csv` DataFrame into memory.

        Returns:
            None
        '''
        if self.verbose:
            logger.info('##########################################################')
            logger.info('#        Pre-processing installments_payments.csv        #')
            logger.info('##########################################################')
            logger.info("Loading the DataFrame, installments_payments.csv, into memory...")

        self.installments_payments = reduce_memory_usage(pd.read_csv(self.file_directory + 'installments_payments.csv', nrows=self.nrows))
        self.initial_shape = self.installments_payments.shape

        if self.verbose:
            logger.info("Loaded installments_payments.csv")
            logger.info('Time Taken to load: {}', datetime.now() - self.start)

    def data_preprocessing_and_feature_engineering(self):
        '''
        Performs preprocessing and feature engineering on the DataFrame.

        Returns:
            None
        '''
        if self.verbose:
            start = datetime.now()
            logger.info("Starting Data Pre-processing and Feature Engineering...")

        # Example of preprocessing and feature engineering (customize as needed)
        # self.installments_payments['new_feature'] = self.installments_payments['feature_1'] / self.installments_payments['feature_2']

        if self.verbose:
            logger.info("Data Pre-processing and Feature Engineering Done.")
            logger.info('Time Taken: {}', datetime.now() - start)

    def aggregations_sk_id_curr(self) -> pd.DataFrame:
        '''
        Aggregates the installments payments on previous loans over SK_ID_CURR.

        Returns:
            pd.DataFrame: Installments payments aggregated over SK_ID_CURR.
        '''
        if self.verbose:
            logger.info("Aggregating installments payments over SK_ID_CURR...")

        # Combining numerical features (only numerical features)
        installments_payments_aggregated = self.installments_payments.select_dtypes(include=[np.number]).drop('SK_ID_PREV', axis=1).groupby(by=['SK_ID_CURR']).mean().reset_index()
        installments_payments_aggregated.columns = ['INSTA_' + column if column != 'SK_ID_CURR' else column for column in installments_payments_aggregated.columns]
        installments_payments_aggregated.fillna(0, inplace=True)

        if self.verbose:
            logger.info('Aggregation Done.')
            logger.info('Size after aggregation: {}', installments_payments_aggregated.shape)

        return installments_payments_aggregated

    def main(self) -> pd.DataFrame:
        '''
        Performs complete preprocessing and aggregation of the `installments_payments` table.

        Returns:
            pd.DataFrame: Final preprocessed and aggregated `installments_payments` table.
        '''
        # Loading the DataFrame
        self.load_dataframe()

        # Performing preprocessing and feature engineering
        self.data_preprocessing_and_feature_engineering()

        # Aggregating the installments payments over SK_ID_CURR
        installments_payments_aggregated = self.aggregations_sk_id_curr()

        if self.verbose:
            logger.info('Done preprocessing installments_payments.')
            logger.info('Initial Size of installments_payments: {}', self.initial_shape)
            logger.info('Size of installments_payments after Pre-Processing, Feature Engineering and Aggregation: {}', installments_payments_aggregated.shape)
            logger.info('Total Time Taken: {}', datetime.now() - self.start)

        if self.dump_to_pickle:
            if self.verbose:
                logger.info('Pickling pre-processed installments_payments to installments_payments_preprocessed.pkl')
            with open(self.file_directory + 'installments_payments_preprocessed.pkl', 'wb') as f:
                pickle.dump(installments_payments_aggregated, f)
            if self.verbose:
                logger.info('Pickling completed.')

        return installments_payments_aggregated

In [ ]:
installments_aggregated = preprocess_installments_payments(file_directory = DIR).main()

2024-08-06 22:35:11.066 | INFO     | __main__:__init__:29 - Preprocessing class initialized.
2024-08-06 22:35:11.070 | INFO     | __main__:load_dataframe:39 - ##########################################################
2024-08-06 22:35:11.073 | INFO     | __main__:load_dataframe:40 - #        Pre-processing installments_payments.csv        #
2024-08-06 22:35:11.074 | INFO     | __main__:load_dataframe:41 - ##########################################################
2024-08-06 22:35:11.076 | INFO     | __main__:load_dataframe:42 - Loading the DataFrame, installments_payments.csv, into memory...
2024-08-06 22:35:28.972 | INFO     | __main__:reduce_memory_usage:24 - Memory usage of dataframe is 830.41 MB
2024-08-06 22:35:30.085 | INFO     | __main__:reduce_memory_usage:55 - Memory usage after optimization is: 311.40 MB
2024-08-06 22:35:30.086 | INFO     | __main__:reduce_memory_usage:56 - Decreased by 62.5%
2024-08-06 22:35:30.089 | INFO     | __main__:load_dataframe:48 - Loaded installment

## POS_CASH_balance

In [ ]:
class preprocess_POS_CASH_balance:
    '''
    Preprocess the POS_CASH_balance table.

    This class contains methods to load, preprocess, and aggregate the `POS_CASH_balance` table.

    Attributes:
        file_directory (str): Path to the directory containing the data files.
        verbose (bool): Whether to enable verbose logging.
        dump_to_pickle (bool): Whether to pickle the final preprocessed table.
        nrows (Optional[int]): Number of rows to read from the CSV file.
    '''

    def __init__(self, file_directory: str = '', verbose: bool = True, dump_to_pickle: bool = False, nrows: Optional[int] = None):
        '''
        Initializes the preprocess_POS_CASH_balance class.

        Args:
            file_directory (str): Path to the directory where the files are located.
            verbose (bool): Whether to enable verbose logging.
            dump_to_pickle (bool): Whether to pickle the final preprocessed table.
            nrows (Optional[int]): Number of rows to read from the CSV file.
        '''
        self.file_directory = file_directory
        self.verbose = verbose
        self.dump_to_pickle = dump_to_pickle
        self.nrows = nrows
        self.start = datetime.now()
        logger.info('Preprocessing class initialized.')

    def load_dataframe(self):
        '''
        Loads the `POS_CASH_balance.csv` DataFrame into memory.

        Returns:
            None
        '''
        if self.verbose:
            logger.info('#########################################################')
            logger.info('#          Pre-processing POS_CASH_balance.csv          #')
            logger.info('#########################################################')
            logger.info("Loading the DataFrame, POS_CASH_balance.csv, into memory...")

        self.pos_cash = reduce_memory_usage(pd.read_csv(self.file_directory + 'POS_CASH_balance.csv', nrows=self.nrows))
        self.initial_size = self.pos_cash.shape

        if self.verbose:
            logger.info("Loaded POS_CASH_balance.csv")
            logger.info('Time Taken to load: {}', datetime.now() - self.start)

    def data_preprocessing_and_feature_engineering(self):
        '''
        Performs preprocessing and feature engineering on the DataFrame.

        Returns:
            None
        '''
        if self.verbose:
            start = datetime.now()
            logger.info("Starting Data Pre-processing and Feature Engineering...")

        # Example of preprocessing and feature engineering (customize as needed)
        # self.pos_cash['new_feature'] = self.pos_cash['feature_1'] / self.pos_cash['feature_2']

        if self.verbose:
            logger.info("Data Pre-processing and Feature Engineering Done.")
            logger.info('Time Taken: {}', datetime.now() - start)

    def aggregations_sk_id_curr(self) -> pd.DataFrame:
        '''
        Aggregates the POS_CASH_balance table over SK_ID_CURR.

        Returns:
            pd.DataFrame: POS_CASH_balance table aggregated over SK_ID_CURR.
        '''
        if self.verbose:
            logger.info("Aggregating POS_CASH_balance over SK_ID_CURR...")

        # Combining numerical features
        pos_cash_numerical_aggregated = self.pos_cash.select_dtypes(include=[np.number]).drop('SK_ID_PREV', axis=1).groupby(by=['SK_ID_CURR']).mean().reset_index()

        # Combining categorical features
        pos_cash_categorical = pd.get_dummies(self.pos_cash.select_dtypes('object'))
        pos_cash_categorical['SK_ID_CURR'] = self.pos_cash['SK_ID_CURR']
        pos_cash_categorical_aggregated = pos_cash_categorical.groupby('SK_ID_CURR').mean().reset_index()

        # Merge numerical and categorical features
        pos_cash_aggregated = pos_cash_numerical_aggregated.merge(pos_cash_categorical_aggregated, on='SK_ID_CURR')
        pos_cash_aggregated.columns = ['POS_' + column if column != 'SK_ID_CURR' else column for column in pos_cash_aggregated.columns]
        pos_cash_aggregated.fillna(0, inplace=True)

        if self.verbose:
            logger.info('Aggregation Done.')
            logger.info('Size after aggregation: {}', pos_cash_aggregated.shape)

        return pos_cash_aggregated

    def main(self) -> pd.DataFrame:
        '''
        Performs complete preprocessing and aggregation of the `POS_CASH_balance` table.

        Returns:
            pd.DataFrame: Final preprocessed and aggregated `POS_CASH_balance` table.
        '''
        # Loading the DataFrame
        self.load_dataframe()

        # Performing preprocessing and feature engineering
        self.data_preprocessing_and_feature_engineering()

        # Aggregating the POS_CASH_balance over SK_ID_CURR
        pos_cash_aggregated = self.aggregations_sk_id_curr()

        if self.verbose:
            logger.info('Done preprocessing POS_CASH_balance.')
            logger.info('Initial Size of POS_CASH_balance: {}', self.initial_size)
            logger.info('Size of POS_CASH_balance after Pre-Processing, Feature Engineering and Aggregation: {}', pos_cash_aggregated.shape)
            logger.info('Total Time Taken: {}', datetime.now() - self.start)

        if self.dump_to_pickle:
            if self.verbose:
                logger.info('Pickling pre-processed POS_CASH_balance to POS_CASH_balance_preprocessed.pkl')
            with open(self.file_directory + 'POS_CASH_balance_preprocessed.pkl', 'wb') as f:
                pickle.dump(pos_cash_aggregated, f)
            if self.verbose:
                logger.info('Pickling completed.')

        return pos_cash_aggregated

In [ ]:
pos_aggregated = preprocess_POS_CASH_balance(file_directory = DIR).main()

2024-08-06 22:35:32.370 | INFO     | __main__:__init__:29 - Preprocessing class initialized.
2024-08-06 22:35:32.373 | INFO     | __main__:load_dataframe:39 - #########################################################
2024-08-06 22:35:32.377 | INFO     | __main__:load_dataframe:40 - #          Pre-processing POS_CASH_balance.csv          #
2024-08-06 22:35:32.378 | INFO     | __main__:load_dataframe:41 - #########################################################
2024-08-06 22:35:32.381 | INFO     | __main__:load_dataframe:42 - Loading the DataFrame, POS_CASH_balance.csv, into memory...
2024-08-06 22:35:43.010 | INFO     | __main__:reduce_memory_usage:24 - Memory usage of dataframe is 610.43 MB
2024-08-06 22:35:43.624 | INFO     | __main__:reduce_memory_usage:55 - Memory usage after optimization is: 238.45 MB
2024-08-06 22:35:43.626 | INFO     | __main__:reduce_memory_usage:56 - Decreased by 60.9%
2024-08-06 22:35:43.628 | INFO     | __main__:load_dataframe:48 - Loaded POS_CASH_balance.cs

## credit_card_balance

In [ ]:
class preprocess_credit_card_balance:
    '''
    Preprocess the credit_card_balance table.

    This class contains methods to load, preprocess, and aggregate the `credit_card_balance` table.

    Attributes:
        file_directory (str): Path to the directory containing the data files.
        verbose (bool): Whether to enable verbose logging.
        dump_to_pickle (bool): Whether to pickle the final preprocessed table.
        nrows (Optional[int]): Number of rows to read from the CSV file.
    '''

    def __init__(self, file_directory: str = '', verbose: bool = True, dump_to_pickle: bool = False, nrows: Optional[int] = None):
        '''
        Initializes the preprocess_credit_card_balance class.

        Args:
            file_directory (str): Path to the directory where the files are located.
            verbose (bool): Whether to enable verbose logging.
            dump_to_pickle (bool): Whether to pickle the final preprocessed table.
            nrows (Optional[int]): Number of rows to read from the CSV file.
        '''
        self.file_directory = file_directory
        self.verbose = verbose
        self.dump_to_pickle = dump_to_pickle
        self.nrows = nrows
        self.start = datetime.now()
        logger.info('Preprocessing class initialized.')

    def load_dataframe(self):
        '''
        Loads the `credit_card_balance.csv` DataFrame into memory.

        Returns:
            None
        '''
        if self.verbose:
            logger.info('#########################################################')
            logger.info('#        Pre-processing credit_card_balance.csv         #')
            logger.info('#########################################################')
            logger.info("Loading the DataFrame, credit_card_balance.csv, into memory...")

        self.cc_balance = pd.read_csv(self.file_directory + 'credit_card_balance.csv', nrows=self.nrows)
        self.initial_size = self.cc_balance.shape

        if self.verbose:
            logger.info("Loaded credit_card_balance.csv")
            logger.info('Time Taken to load: {}', datetime.now() - self.start)

    def data_preprocessing_and_feature_engineering(self):
        '''
        Performs preprocessing and feature engineering on the DataFrame.

        Returns:
            None
        '''
        if self.verbose:
            start = datetime.now()
            logger.info("Starting Data Pre-processing and Feature Engineering...")

        # Example of preprocessing and feature engineering (customize as needed)
        # self.cc_balance['new_feature'] = self.cc_balance['feature_1'] / self.cc_balance['feature_2']

        if self.verbose:
            logger.info("Data Pre-processing and Feature Engineering Done.")
            logger.info('Time Taken: {}', datetime.now() - start)

    def aggregations(self) -> pd.DataFrame:
        '''
        Aggregates the `credit_card_balance` table first over `SK_ID_PREV`, and then over `SK_ID_CURR`.

        Returns:
            pd.DataFrame: Aggregated `credit_card_balance` table.
        '''
        if self.verbose:
            logger.info("Aggregating the DataFrame, first over SK_ID_PREV, then over SK_ID_CURR")

        # Combining numerical features
        cc_numerical_aggregated = self.cc_balance.select_dtypes(include=[np.number]).drop('SK_ID_PREV', axis=1).groupby(by=['SK_ID_CURR']).mean().reset_index()

        # Combining categorical features
        cc_categorical = pd.get_dummies(self.cc_balance.select_dtypes('object'))
        cc_categorical['SK_ID_CURR'] = self.cc_balance['SK_ID_CURR']
        cc_categorical_aggregated = cc_categorical.groupby('SK_ID_CURR').mean().reset_index()

        # Merge numerical and categorical features
        cc_aggregated = cc_numerical_aggregated.merge(cc_categorical_aggregated, on='SK_ID_CURR')
        cc_aggregated.columns = ['CC_' + column if column != 'SK_ID_CURR' else column for column in cc_aggregated.columns]
        cc_aggregated.fillna(0, inplace=True)

        if self.verbose:
            logger.info('Aggregation Done.')
            logger.info('Size after aggregation: {}', cc_aggregated.shape)

        return cc_aggregated

    def main(self) -> pd.DataFrame:
        '''
        Performs complete preprocessing and aggregation of the `credit_card_balance` table.

        Returns:
            pd.DataFrame: Final preprocessed and aggregated `credit_card_balance` table.
        '''
        # Loading the DataFrame
        self.load_dataframe()

        # Performing preprocessing and feature engineering
        self.data_preprocessing_and_feature_engineering()

        # Aggregating the `credit_card_balance` over SK_ID_PREV and SK_ID_CURR
        cc_aggregated = self.aggregations()

        if self.verbose:
            logger.info('Done preprocessing credit_card_balance.')
            logger.info('Initial Size of credit_card_balance: {}', self.initial_size)
            logger.info('Size of credit_card_balance after Pre-Processing, Feature Engineering and Aggregation: {}', cc_aggregated.shape)
            logger.info('Total Time Taken: {}', datetime.now() - self.start)

        if self.dump_to_pickle:
            if self.verbose:
                logger.info('Pickling pre-processed credit_card_balance to credit_card_balance_preprocessed.pkl')
            with open(self.file_directory + 'credit_card_balance_preprocessed.pkl', 'wb') as f:
                pickle.dump(cc_aggregated, f)
            if self.verbose:
                logger.info('Pickling completed.')

        return cc_aggregated

In [ ]:
cc_aggregated = preprocess_credit_card_balance(file_directory=DIR).main()

2024-08-06 22:35:49.704 | INFO     | __main__:__init__:29 - Preprocessing class initialized.
2024-08-06 22:35:49.707 | INFO     | __main__:load_dataframe:39 - #########################################################
2024-08-06 22:35:49.709 | INFO     | __main__:load_dataframe:40 - #        Pre-processing credit_card_balance.csv         #
2024-08-06 22:35:49.711 | INFO     | __main__:load_dataframe:41 - #########################################################
2024-08-06 22:35:49.713 | INFO     | __main__:load_dataframe:42 - Loading the DataFrame, credit_card_balance.csv, into memory...
2024-08-06 22:36:03.884 | INFO     | __main__:load_dataframe:48 - Loaded credit_card_balance.csv
2024-08-06 22:36:03.886 | INFO     | __main__:load_dataframe:49 - Time Taken to load: 0:00:14.182208
2024-08-06 22:36:03.890 | INFO     | __main__:data_preprocessing_and_feature_engineering:60 - Starting Data Pre-processing and Feature Engineering...
2024-08-06 22:36:03.893 | INFO     | __main__:data_preproc

## application_train and application_test

In [ ]:
class preprocess_application_train_test:
    '''
    Preprocess the application_train and application_test tables.
    Contains 4 member functions:
        1. init method
        2. load_dataframes method
        3. data_cleaning method
        4. main method
    '''

    def __init__(self, file_directory1='', file_directory2='', verbose=True, dump_to_pickle=False):
        '''
        Initialize the class members.

        Args:
            file_directory1: str, default=''
                Path where the application_train.csv file exists. Include a '/' at the end.
            file_directory2: str, default=''
                Path where the application_test.csv file exists. Include a '/' at the end.
            verbose: bool, default=True
                Whether to enable verbosity or not.
            dump_to_pickle: bool, default=False
                Whether to pickle the final preprocessed tables or not.

        '''
        self.verbose = verbose
        self.dump_to_pickle = dump_to_pickle
        self.file_directory1 = file_directory1
        self.file_directory2 = file_directory2

    def load_dataframes(self):
        '''
        Load the application_train.csv and application_test.csv DataFrames.

        Returns:
            None
        '''
        if self.verbose:
            self.start = datetime.now()
            logger.info('#######################################################')
            logger.info('#        Pre-processing application_train.csv         #')
            logger.info('#        Pre-processing application_test.csv          #')
            logger.info('#######################################################')
            logger.info("\nLoading the DataFrames into memory...")

        self.application_train = reduce_memory_usage(pd.read_csv(self.file_directory1 + 'application_train.csv'))
        self.application_test = reduce_memory_usage(pd.read_csv(self.file_directory2 + 'application_test.csv'))
        self.initial_train_shape = self.application_train.shape
        self.initial_test_shape = self.application_test.shape

        if self.verbose:
            logger.info("Loaded application_train.csv and application_test.csv")
            logger.info(f"Time Taken to load = {datetime.now() - self.start}")

    def data_cleaning(self):
        '''
        Clean the tables by removing erroneous rows/entries.

        Returns:
            None
        '''
        if self.verbose:
            logger.info("\nPerforming Data Cleaning...")
        #there are some FLAG_DOCUMENT features having just one category for almost all data, we will remove those
        flag_cols_to_drop = ['FLAG_DOCUMENT_2', 'FLAG_DOCUMENT_4', 'FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_12', 'FLAG_DOCUMENT_20']

        self.application_train.drop(flag_cols_to_drop, axis=1, inplace=True)
        self.application_test.drop(flag_cols_to_drop, axis=1, inplace=True)

        if self.verbose:
            logger.info("Data Cleaning Done.")

    def main(self):
        '''
        Complete preprocessing of application_train and application_test tables.

        Returns:
            Final preprocessed application_train and application_test tables.
        '''
        # Load the DataFrames
        self.load_dataframes()

        # Perform Data Cleaning
        self.data_cleaning()

        # Feature Engineering
        if self.verbose:
            start = datetime.now()
            logger.info("\nStarting Feature Engineering...")
            logger.info("\nCreating Domain Based Features on Numeric Data")

        cat_col_train = [category for category in self.application_train.columns if self.application_train[category].dtype == 'object']
        cat_col_test = [category for category in self.application_test.columns if self.application_test[category].dtype == 'object']

        self.application_train = pd.get_dummies(self.application_train, columns=cat_col_train)
        self.application_test = pd.get_dummies(self.application_test, columns=cat_col_test)

        if self.verbose:
            logger.info("Creating features based on Categorical Interactions on some Numeric Features")
            logger.info("Feature Engineering Done.")
            logger.info(f"Time taken = {datetime.now() - start}")

        if self.verbose:
            logger.info('Preprocessing Done.')
            logger.info(f"\nInitial Size of application_train: {self.initial_train_shape}")
            logger.info(f"Size of application_train after Pre-Processing and Feature Engineering: {self.application_train.shape}")
            logger.info(f"\nInitial Size of application_test: {self.initial_test_shape}")
            logger.info(f"Size of application_test after Pre-Processing and Feature Engineering: {self.application_test.shape}")
            logger.info(f'\nTotal Time Taken = {datetime.now() - self.start}')

        if self.dump_to_pickle:
            if self.verbose:
                logger.info('\nPickling pre-processed application_train and application_test to pickle files.')
            with open(self.file_directory1 + 'application_train_preprocessed.pkl', 'wb') as f:
                pickle.dump(self.application_train, f)
            with open(self.file_directory2 + 'application_test_preprocessed.pkl', 'wb') as f:
                pickle.dump(self.application_test, f)
            if self.verbose:
                logger.info('Pickling Done.')
        if self.verbose:
            logger.info('-'*100)

        return self.application_train, self.application_test

In [ ]:
application_train, application_test = preprocess_application_train_test(file_directory1=CLEANED_DATA_DIR, file_directory2=DIR).main()

2024-08-06 22:36:07.141 | INFO     | __main__:load_dataframes:40 - #######################################################
2024-08-06 22:36:07.143 | INFO     | __main__:load_dataframes:41 - #        Pre-processing application_train.csv         #
2024-08-06 22:36:07.149 | INFO     | __main__:load_dataframes:42 - #        Pre-processing application_test.csv          #
2024-08-06 22:36:07.151 | INFO     | __main__:load_dataframes:43 - #######################################################
2024-08-06 22:36:07.153 | INFO     | __main__:load_dataframes:44 - 
Loading the DataFrames into memory...
2024-08-06 22:36:14.047 | INFO     | __main__:reduce_memory_usage:24 - Memory usage of dataframe is 286.23 MB
2024-08-06 22:36:14.751 | INFO     | __main__:reduce_memory_usage:55 - Memory usage after optimization is: 91.79 MB
2024-08-06 22:36:14.754 | INFO     | __main__:reduce_memory_usage:56 - Decreased by 67.9%
2024-08-06 22:36:15.987 | INFO     | __main__:reduce_memory_usage:24 - Memory usage of

In [ ]:
import re
def merge_all_tables(application_train, application_test, bureau_aggregated, previous_aggregated,
                    installments_aggregated, pos_aggregated, cc_aggregated):
    '''
    Function to merge all the tables together with the application_train and application_test tables
    on SK_ID_CURR.

    Inputs:
        All the previously pre-processed Tables.

    Returns:
        Single merged tables, one for training data and one for test data
    '''
    logger.info("Merging application_train and application_test with aggregated tables.")

    #merging application_train and application_test with Aggregated bureau table
    app_train_merged = application_train.merge(bureau_aggregated, on = 'SK_ID_CURR', how = 'left')
    app_test_merged = application_test.merge(bureau_aggregated, on = 'SK_ID_CURR', how = 'left')
    logger.info("Merged with bureau_aggregated.")

    #merging with aggregated previous_applications
    app_train_merged = app_train_merged.merge(previous_aggregated, on = 'SK_ID_CURR', how = 'left')
    app_test_merged = app_test_merged.merge(previous_aggregated, on = 'SK_ID_CURR', how = 'left')
    logger.info("Merged with previous_aggregated.")

    #merging with aggregated installments tables
    app_train_merged = app_train_merged.merge(installments_aggregated, on = 'SK_ID_CURR', how = 'left')
    app_test_merged = app_test_merged.merge(installments_aggregated, on = 'SK_ID_CURR', how = 'left')
    logger.info("Merged with installments_aggregated.")

    #merging with aggregated POS_Cash balance table
    app_train_merged = app_train_merged.merge(pos_aggregated, on = 'SK_ID_CURR', how = 'left')
    app_test_merged = app_test_merged.merge(pos_aggregated, on = 'SK_ID_CURR', how = 'left')
    logger.info("Merged with pos_aggregated.")

    #merging with aggregated credit card table
    app_train_merged = app_train_merged.merge(cc_aggregated, on = 'SK_ID_CURR', how = 'left')
    app_test_merged = app_test_merged.merge(cc_aggregated, on = 'SK_ID_CURR', how = 'left')
    logger.info("Merged with cc_aggregated.")

    # Filling missing values with 0
    app_train_merged = app_train_merged.fillna(0)
    app_test_merged = app_test_merged.fillna(0)
    logger.info("Filled missing values with 0.")

    # Clean column names to remove non-alphanumeric characters
    app_train_merged = app_train_merged.rename(columns=lambda x: re.sub('[^A-Za-z0-9_]+', '', x))
    app_test_merged = app_test_merged.rename(columns=lambda x: re.sub('[^A-Za-z0-9_]+', '', x))
    logger.info("Cleaned column names.")

    # Ensure the columns are the same for train and test data
    train_cols = set(app_train_merged.columns)
    test_cols = set(app_test_merged.columns)

    diff_train_cols = train_cols.difference(test_cols)
    diff_test_cols = test_cols.difference(train_cols)

    if 'TARGET' in diff_train_cols:
        diff_train_cols.remove('TARGET')

    app_train_merged.drop(diff_train_cols, axis=1, inplace=True)
    app_test_merged.drop(diff_test_cols, axis=1, inplace=True)
    logger.info("Ensured train and test data have the same columns.")


    #removing the SK_ID_CURR from training  data
    app_train_merged = app_train_merged.drop(['SK_ID_CURR'], axis = 1)
    logger.info("Removed SK_ID_CURR from the data.")

    return reduce_memory_usage(app_train_merged), reduce_memory_usage(app_test_merged)

In [ ]:
train_data, test_data = merge_all_tables(application_train, application_test, bureau_aggregated, previous_aggregated,
                    installments_aggregated, pos_aggregated, cc_aggregated)

2024-08-06 22:48:55.333 | INFO     | __main__:reduce_memory_usage:24 - Memory usage of dataframe is 650.17 MB
2024-08-06 22:48:58.283 | INFO     | __main__:reduce_memory_usage:55 - Memory usage after optimization is: 295.03 MB
2024-08-06 22:48:58.286 | INFO     | __main__:reduce_memory_usage:56 - Decreased by 54.6%
2024-08-06 22:48:58.316 | INFO     | __main__:reduce_memory_usage:24 - Memory usage of dataframe is 103.29 MB
2024-08-06 22:48:59.116 | INFO     | __main__:reduce_memory_usage:55 - Memory usage after optimization is: 47.00 MB
2024-08-06 22:48:59.118 | INFO     | __main__:reduce_memory_usage:56 - Decreased by 54.5%


In [ ]:
train_data.to_csv(DIR + 'train_data_final.csv')
test_data.to_csv(DIR + 'test_data_final.csv')

In [ ]:
!kaggle competitions download -c home-credit-default-risk

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 7, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 407, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.config/kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/
